# Exploring SAS file metadata

Anyone working with the OAI structured data has a choice, import the data from ASCII or SAS. This project chooses to rely on the SAS data. There are over 9,000 different variables recorded by OAI.  Writing heuristics to guess the optimal data types for all 9,000 is likely to be more flawed than leveraging what we can from SAS metadata. This notebook explores what SAS metadata can be pulled out by pyreadstat (there may be metadata it ignores; I haven't verified it's code). It is mostly here as a record of discovery and not typically needed for anyone looking to jump into the data.

The data seems to be stored in two ways:
* A collection of sas7bdat and sas7bcat files.
* In the SAS propietary CPORT format (labeled .xpt instead of .cpt)

Thanks to the OAI employee who chose to support a closed source, proprietary format. While SAS was common enough in 2012, chosing proprietary formats for govt. owned data was already bad form by then. Further, the files are listed as .XPT files just to keep users confused (you can find users trying to solve this mystery for this exact dataset in internet forums).  Not having a SAS instance, we have to ignore the the CPORT files and hope no information is lost in doing so. It isn't also clear why the data is saved in a compressed format (save space) but also bundled with a non-compressed form (benefits of compression lost). Maybe historical reasons.

Subtle details may change between OAI release versions. Thus, once facts are established, they are encoded as assertions that can be verified when a new version of the data is released.

## Setup / Imports / Constants

In [ ]:
# Setup 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))

In [ ]:
import math
import os
import pandas as pd
import pickle
import pyreadstat
from tqdm import tqdm
import datetime
import re

In [ ]:
# Constants
data_dir = '../data/structured_data/'
pdfs_dir = '../data/pdfs/General/Formats_SAS/'

# Metadata values pulled out by pyreadstat
meta_vars = [ 'column_labels',
 'column_names',
 'column_names_to_labels',
 'file_encoding',
 'file_format',
 'file_label',
 'missing_ranges',
 'missing_user_values',
 'notes',
 'number_columns',
 'number_rows',
 'original_variable_types',
 'readstat_variable_types',
 'table_name',
 'value_labels',
 'variable_alignment',
 'variable_display_width',
 'variable_measure',
 'variable_storage_width',
 'variable_to_label',
 'variable_value_labels']

## Read in all metadata

In [ ]:
# All SAS files
all_files = os.listdir(data_dir)
all_files = [x for x in all_files if '.sas7bdat' in x]
all_files.remove('sageancillarystudy_formats.sas7bdat') ## At a binary level this seems like another CPORT file. WTF?
all_files.sort()

In [ ]:
files_meta = {}
for filename in all_files:
    meta_dict['Filename'].append(filename)
    _, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + filename,
                                                         catalog_file=data_dir + 'formats.sas7bcat',
                                                         num_processes=6, metadataonly=True)
    files_meta[filename] = meta

### See what metadata variable values are common across all files

In [ ]:
# See what metadata is collected across all files
# if a single value, list it.
# if a collection store the size

# create storage dict
meta_dict = {v: [] for v in meta_vars}
meta_dict['Filename'] = []

for file, meta in files_meta.items():
    meta_dict['Filename'].append(filename)

    for mv in meta_vars:
        var = getattr(meta, mv)
        if var:
            if isinstance(var, int):
                meta_dict[mv].append(var)
            elif isinstance(var, str):
                meta_dict[mv].append(var)
            else:
                meta_dict[mv].append(len(var))
        else:
             meta_dict[mv].append(None)

df = pd.DataFrame(meta_dict).set_index('Filename')
df

In [ ]:
# Sanity checks on columns that should all have the same number of values and same keys in their dicts
# - failure means the data (or pyreaadstat) has changed since this was written
for file, meta in files_meta.items():
    assert meta.number_columns == len(meta.column_labels)
    assert meta.number_columns == len(meta.column_names)
    # Do column_names + column_labels = column_names_to_labels?
    assert set(meta.column_labels) == set(meta.column_names_to_labels.values())
    
    # Do all variables match a column name?
    names = set(meta.column_names)
    assert len(names ^ set(meta.column_names_to_labels.keys())) == 0
    assert len(names ^ set(meta.original_variable_types.keys())) == 0
    assert len(names ^ set(meta.readstat_variable_types.keys())) == 0
    assert len(names ^ set(meta.variable_alignment.keys())) == 0
    assert len(names ^ set(meta.variable_display_width.keys())) == 0
    assert len(names ^ set(meta.variable_storage_width.keys())) == 0
    assert names >= set(meta.variable_to_label.keys())

In [ ]:
# List metadata variables which are empty for all files

print(str(list(df.columns[df.isna().all()].sort_values())))

In [ ]:
# Sanity check
# - failure means the data (or pyreaadstat) has changed since this was written
empty_vars = ['missing_ranges', 'missing_user_values', 'notes', 'value_labels', 'variable_value_labels']
for var in empty_vars:
    assert df[var].isna().all()

In [ ]:
# Which columns have an occasional empty value?

incomplete = set(df.columns[df.isna().any()]) - set(df.columns[df.isna().all()])
for x in incomplete:
    print(x + ': ' + str(df[x].unique()))

In [ ]:
# How many unique valeues exist for the remaining metadata variables?
for col in df.columns[~df.isna().any()]:
    print(col + ': ' + str(len(df[col].unique())))

So `file_encoding`, `file_format` are the same across all files.

In [ ]:
# Sanity check
# - failure means the data (or pyreaadstat) has changed since this was written
assert len(list(df['file_encoding'].unique())) == 1
assert list(df['file_encoding'].unique())[0] == 'WINDOWS-1252'
assert len(list(df['file_format'].unique())) == 1
assert list(df['file_format'].unique())[0] == 'sas7bdat'

In [ ]:
# See what the unique values occur for given variable dictionaries
col_list = ['original_variable_types', 'readstat_variable_types', 'variable_alignment', 'variable_display_width', 'variable_measure', 'variable_storage_width', 'variable_to_label']

col_sets = {c: set() for c in col_list}
for file, meta in files_meta.items():
    for col in col_list:
        var = getattr(meta, col)
        if var:
            if isinstance(var, dict):
                col_sets[col].update(var.values())
            else:
                col_sets[col].update(var)
                
for col, vals in col_sets.items():
    print(col + '(' + str(len(vals)) + '): ' + str(list(vals)) + '\n')

In [ ]:
col_sets['original_variable_types'] - col_sets['variable_to_label']

So it seems that only `original_variable_types`, `variable_storage_width`, `variable_to_label` have anything unique to say about a variable (aside from variable/column names and labels). As shown several cells above, `original_variable_types` has a dict value for every variable (even if that value is `NULL`). `variable_to_label` seems to have the same data, but leaves out variables that don't use a value list. 'Value labels' seem to be SAS's form of user defined formats ( https://libguides.library.kent.edu/SAS/UserDefinedFormats ), somewhat like categories in Pandas.

In [ ]:
# Sanity check
# - failure means the data (or pyreaadstat) has changed since this was written
assert col_sets['readstat_variable_types'] == {'double', 'string'}
assert col_sets['variable_alignment'] == {'unknown'}
assert col_sets['variable_display_width'] == {0}
assert col_sets['variable_measure'] == {'unknown'}

## Check for differences in format files

There seem to be two competing SAS catalog files*:
* `../data/pdfs/General/Formats_SAS/formats.sas7bcat`
* `../data/structured_data/formats.sas7bcat`

At a binary level, they are different, let's look at them as far as pyreadstat can determine. 

*(Ignoring the `kMRI_SQ_WORMS_Link_Formats.sas7bcat` file for now)

In [ ]:
_, local_cat = pyreadstat.read_sas7bcat(data_dir + 'formats.sas7bcat')
_, pdf_cat = pyreadstat.read_sas7bcat(pdfs_dir + 'formats.sas7bcat')

for cat in [local_cat, pdf_cat]:
    for v in meta_vars:
        var = getattr(cat,v)
        if var:
            if isinstance(var, int):
                print(v + ': ' + str(var))
            else:
                print(v + ' - ' + str(len(var)))
        else:
            print(v + ' - empty')
    print()

They seem the same. Both only contain 'value_labels', SAS's user defined categories. Confirm:

In [ ]:
# Compare the dicts of the local catalog vs the one in the pdfs dir
# - each is a dict of value dicts
# - this is more code than normally needed because pyreadstat is creating NaN keys

# At the top level, both have the same keys
assert set(local_cat.value_labels.keys()) == set(pdf_cat.value_labels.keys())

for k1, v1 in local_cat.value_labels.items():
    # Get both sub dictionaries
    v1_list = [(k,v) for k,v in v1.items()]
    pdf_v1_list = [(k,v) for k,v in pdf_cat.value_labels[k1].items()]
        
    for (k2, v2) in v1_list:
        # Confirm match exists
        key_match = False
        for (pdf_k2, pdf_v2) in pdf_v1_list:
            if pdf_k2 == k2 or (isinstance(k2, float) and math.isnan(k2) and math.isnan(pdf_k2)):
                key_match = True
                if pdf_v2 == v2:
                    break
                else:
                    # Mis-match on sub-dict values
                    print('Dict[' + str(k1) + '][' + str(k2) + ']: [local v: ' + v2 + ']\t[pdf v: ' + pdf_v2 + ']')
                    break
        if not key_match:
            print('Dict[' + str(k1) + '): [local k: ' + str(k2) + ' had no match in: ' +  str([k for (k,v) in pdf_v1_list]))

In [ ]:
# Flip the order see if any keys exist in the PDF versions that don't exist in the local catalog files

for k1, v1 in pdf_cat.value_labels.items():
    # Get both sub dictionaries
    v1_list = [(k,v) for k,v in v1.items()]
    local_v1_list = [(k,v) for k,v in local_cat.value_labels[k1].items()]
        
    for (k2, v2) in v1_list:
        # Confirm match exists
        key_match = False
        for (local_k2, local_v2) in local_v1_list:
            if local_k2 == k2 or (isinstance(k2, float) and math.isnan(k2) and math.isnan(local_k2)):
                key_match = True
                if local_v2 == v2:
                    break
        if not key_match:
            print('Dict[' + str(k1) + '): [pdf k: ' + str(k2) + ' had no match in: ' +  str([k for (k,v) in local_v1_list]))

So two keys shipped with the data aren't in the catalog file shipped with the PDFs. Seven seem to have minor differences in their label text.

In [ ]:
# Are any of the conflicting value dicts present in this data set?
conflicting_value_labels = {'$SYNACCP', 'WTCHG', 'CRTMRPH', 'MMMRPH'}
col_sets['variable_to_label']

print(conflicting_value_labels & col_sets['variable_to_label'])

### Undefined data formats

In [ ]:
# Are their any variable types that aren't user defined?
undefined_set = col_sets['variable_to_label'] - set(local_cat.value_labels.keys())
print(undefined_set)

* $ in SAS means character data
* BEST is numeric data that lets the system chose the best display format
* MMDDYY is obviously a date

I haven't found documentation for the rest ( https://documentation.sas.com/doc/en/pgmsascdc/9.4_3.5/leforinforref/p0z62k899n6a7wn1r5in6q5253v1.htm ). Formats whose definitions weren't exported?

In [ ]:
# Which data sets contain these undefined formats
undefined_set.remove('$')
undefined_set.remove('BEST')
undefined_set.remove('MMDDYY')


for filename, meta in files_meta.items():
    vls = set(meta.variable_to_label.values())
    for value_label in undefined_set:
        if value_label in vls:
            print(filename + ' ' + str(value_label))

This makes sense. For some reason `kmri_sq_worms` has its own catalog file. Who knows what is going on with the `sageancillarystudy`? Ignoring for now.

### Look closer at NaN keys
This is a side-effect of pyreadstat, not SAS.

In [ ]:
# Look for dictionaries with more than one NaN entry.
for k1, v1 in local_cat.value_labels.items():
    v1_klist = [k for k in v1.keys()]
    cnt = 0    
    for k2 in v1_klist:
        if isinstance(k2, float) and math.isnan(k2):
            cnt += 1
    if cnt > 1:
        print("Duplicate NaN keys in dict: " + k1)

In [ ]:
# See if the count differs for the second catalog
for k1, v1 in pdf_cat.value_labels.items():
    v1_klist = [k for k in v1.keys()]
    cnt = 0    
    for k2 in v1_klist:
        if isinstance(k2, float) and math.isnan(k2):
            cnt += 1
    if cnt > 1:
        print("Duplicate NaN keys in dict: " + k1)

In [ ]:
# When two NaNs occur, what are the values?
for k1, v1 in local_cat.value_labels.items():
    # Get both sub dictionaries
    v1_list = [(k,v) for k,v in v1.items()]
    
    cnt = 0
    last_val = ''
    for (k2, v2) in v1_list:
        if isinstance(k2, float) and math.isnan(k2):
            cnt += 1
            if cnt > 1:
                print(k1 + ': ' + last_val + '\t' + v2)
            last_val = v2

The duplicates seem to be a parsing issue, not two values that parse two NaN.

In [ ]:
# How many value labels contain one or more NaNs?
cnt = 0    
for k1, v1 in local_cat.value_labels.items():
    v1_klist = [k for k in v1.keys()]
    for k2 in v1_klist:
        if isinstance(k2, float) and math.isnan(k2):
            cnt += 1
            break
print("Dicts: " + str(len(local_cat.value_labels)) + '\tDicts w/ NaNs: ' + str(cnt))

Seems that NaN's are common.

## Explore optimal pyreadstat settings

Look at how various pyreadstat flags interact with this particular dataset.

In [ ]:
# Grab a small sample dataset with mixed types (including user defined types)
# Parse with only catalog files
filename = 'kmri_sq_blksbml_bicl03.sas7bdat'
df1, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + filename,
                                                         catalog_file=data_dir + 'formats.sas7bcat',
                                                         num_processes=6)
df1

In [ ]:
# Look at what types pyreadstat chose with a provided catalog file
df1.dtypes

In [ ]:
# Confirm that two columns share a user defined type
files_meta[filename].variable_to_label

In [ ]:
# What possible values?
local_cat.value_labels['BBMLSPE']

In [ ]:
df1.V03BBMLP.value_counts()

In [ ]:
# How many NaN values
df1.V03BBMLP.isna().sum()

In [ ]:
# Parse with catalog files and user_missing=True
df2, meta2 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + filename,
                                                         catalog_file=data_dir + 'formats.sas7bcat',
                                                         num_processes=6, user_missing=True)

In [ ]:
df2.V03BBMLP.value_counts()

In [ ]:
df2.V03BBMLP.isna().sum()

This only leaves the NaNs to be cconverted to the user defined categorical.  It seems pyreadstat parses . as NaN, but only connects NaN to a value label in the value label table. In the actual data it remains a NaN. This could be confusing if NaNs exist in the data for other reasons.

### Looking at dates

In [ ]:
# Parse with catalog files and user_missing=True and dates_as_pandas_datetime=True
filename = 'outcomes99.sas7bdat'
df3, meta3 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sas7bdat, data_dir + filename,
                                                         catalog_file=data_dir + 'formats.sas7bcat',
                                                         num_processes=6, dates_as_pandas_datetime=True, user_missing=True)
df3

* No flags: date columns are mixed objects: NaN (float) and Python datatime.date objects
* If dates_as_pandas_datetime=True, then columns become Pandas.datetimens[64]
* If dates_as_pandas_datetime=True and user_missing=True, get a mixed objects: str and datetime.datetime objects (note strs are missing data flags without the starting .)

In a system in like Pandas you can't combine multiple missing value types with other datatypes. In this case, only one missing value type is used (.A) but that isn't guaranteed for all dates. If this is the only missing value flag, it can be noted in comments that NaT = .A, and all date columns can be converted to Pandas datetime columns.  While this example only shows it in relation to dates, but it could also apply to numeric columns, with more missing types being used.

### How many variables don't have set categories?
Both dates and numeric value columns will need closer inspection to store them well.  How many are there?

In [ ]:
# How many variables don't have a set data format?
var_cnt = 0
null_cnt = 0
date_cnt = 0
for filename, meta in files_meta.items():
    var_cnt += len(meta.original_variable_types.keys())
    null_cnt += len([n for n in meta.original_variable_types.values() if n == 'NULL'])
    date_cnt += len([n for n in meta.original_variable_types.values() if n == 'MMDDYY'])
print('Total variables stored: ' + str(var_cnt))
print('Null variables stored: ' + str(null_cnt))
print('Date variables stored: ' + str(date_cnt))